Objective 2 of the Harvard Data Commons work seeks to make it easier for users of the Harvard Dataverse Repository to find and re-use the "workflow" datasets that the repository hosts. The group working on enhancing the repository's functionality toward this end is defining "workflow" datasets as any datasets containing any of several types of files that indicate that the published data is meant to be computationally reproducible.

For research into the reusability of workflow datasets, see "Trisovic, A., Lau, M.K., Pasquier, T. et al. A large-scale study on research code quality and execution. Sci Data 9, 60 (2022). https://doi.org/10.1038/s41597-022-01143-6"

This notebook aims to provide benchmarks for how findable these workflow datasets have been. And after changes to the repository are made in order to improve their discoverability, the methods explored in this notebook might help indicate the effectiveness of those changes by comparing these measurements to measurements taken after the changes have been in place for some amount of time.

Two possible methods for measuring the discoverability of workflow datasets are to:
- Find the average number of times that the pages of these datasets are viewed
- Find the average number of times that the files of these datasets are downloaded

The Harvard Dataverse uses Google Analytics to track page views, so we can look there for metrics about how often all datasets are viewed. To find which datasets would be considered workflow datasets, as well as to find out how often files in all datasets, including workflow datasets, are downloaded, we can look in the Harvard Dataverse's database.

I think these two sources of data should be enough to determine:
- What has been the average sum of file downloads of all datasets in the repository?
- What has been the average sum of file downloads of workflow datasets?
- On average, how often is each dataset's page viewed?
- On average, how often is each workflow dataset's page viewed?

